In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
from utils import (prepare_data_CNN, create_sequences_CNN, split_data_CNN, build_model_CNN, train_model_CNN, make_predictions_CNN)

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Favorita

In [2]:
# Leitura dos dados tratados
favorita = pd.read_parquet('Bases Tratadas/favorita_item_loja_semana.parquet')
favorita['Data'] = pd.to_datetime(favorita['Data'])

In [3]:
# Agrupamento de vendas por data
favorita_agrupado = favorita[['Data', 'Vendas']].groupby('Data').sum().reset_index()
favorita_agrupado.set_index('Data', inplace=True)

In [4]:
# Preparar os dados
data_scaled, scaler = prepare_data_CNN(favorita_agrupado)

look_back = 10

# Criar sequências e dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test, train_size = split_data_CNN(data_scaled, look_back)


# Construir e treinar o modelo
model = build_model_CNN(look_back)

start_train_time = time.time()
history = train_model_CNN(model, X_train, y_train)
end_train_time = time.time()
train_time = end_train_time - start_train_time


# Realizar previsões no conjunto de teste
start_forecast_time = time.time()
test_predict = make_predictions_CNN(model, X_test)
end_forecast_time = time.time()
forecast_time = end_forecast_time - start_forecast_time


# Reverter a normalização das previsões e dos valores reais
test_predict_original = scaler.inverse_transform(test_predict).flatten()
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

# Obter os índices reais para treino e teste
train_index = favorita_agrupado.index[:train_size]  # Índices do conjunto de treino
test_index = favorita_agrupado.index[train_size:]   # Índices do conjunto de teste

# Dados de treino original
train_original = scaler.inverse_transform(data_scaled[:train_size]).flatten()

# Dados reais para plotagem do teste
data_test_original = scaler.inverse_transform(data_scaled[train_size:]).flatten()

# Plotando os dados originais e previstos
fig = go.Figure()

# Dados reais (treino)
fig.add_trace(go.Scatter(
    x=train_index, 
    y=train_original, 
    mode='lines', 
    name='Treino (Real)', 
    line=dict(color='blue')
))

# Dados reais (teste)
fig.add_trace(go.Scatter(
    x=test_index, 
    y=data_test_original, 
    mode='lines', 
    name='Teste (Real)', 
    line=dict(color='orange')
))

# Dados previstos (teste)
fig.add_trace(go.Scatter(
    x=test_index[:len(test_predict_original)],  # Garantir o alinhamento
    y=test_predict_original, 
    mode='lines', 
    name='Teste (Previsto)', 
    line=dict(color='green')
))

# Configuração do gráfico
fig.update_layout(
    title='Vendas Reais vs Previstas',
    xaxis_title='Data',
    yaxis_title='Vendas',
    legend_title='Legenda',
    template='plotly_white'
)

fig.show()


3/3 [==============================] - 0s 7ms/step


In [5]:
# Criar um dataframe com data, valor real e valor previsto
df_resultado = pd.DataFrame({
    'Data': test_index,
    'Real': y_test_original,
    'Previsto': test_predict_original
})

# Salvar o dataframe em um arquivo CSV
df_resultado.to_csv('Resultados/Previsao/Favorita/Tudo/CNN.csv', index=False)

# Criar um dataframe com o tempo de treino e previsão
df_tempo = pd.DataFrame({
    'Treino': [train_time],
    'Previsao': [forecast_time]
})
# Salvar o dataframe em um arquivo CSV
df_tempo.to_csv('Resultados/Tempo/Favorita/Tudo/CNN.csv', index=False)

In [6]:
lojas_relevantes_favorita = [3, 8, 44, 45, 46, 47, 48, 49, 50, 51]
produtos_relevantes_favorita = {
    3: [1047679, 1503844],
    8: [1503844, 1047679],
    44: [1047679, 1503844],
    45: [1503844, 1473474],
    46: [584028, 1463992],
    47: [1503844, 1047679],
    48: [584028, 1463992],
    49: [1503844, 1473474],
    50: [584028, 364606],
    51: [1503844, 257847]
}

In [7]:
modelos_por_loja = {}
tempos_treinamento = {}
tempos_previsao = {}

# Create a figure
fig = make_subplots(rows=5, cols=2, subplot_titles=[f'Loja {loja}' for loja in lojas_relevantes_favorita])

df_resultado_loja = pd.DataFrame(columns=['Data', 'Loja', 'Real', 'Previsto'])
df_resultado_tempo = pd.DataFrame(columns=['Loja', 'Treino', 'Previsao'])

for i, loja in enumerate(lojas_relevantes_favorita):

    df_loja = favorita[favorita['Loja'] == loja]
    df_loja_grouped = df_loja.groupby('Data')['Vendas'].sum().reset_index()
    df_loja_grouped.set_index('Data', inplace=True)
    data_scaled, scaler = prepare_data_CNN(df_loja_grouped)
    look_back = 10

    X_train, X_test, y_train, y_test, train_size = split_data_CNN(data_scaled, look_back)

    # Construir e treinar o modelo
    model = build_model_CNN(look_back)

    start_train_time = time.time()
    history = train_model_CNN(model, X_train, y_train)
    end_train_time = time.time()
    train_time = end_train_time - start_train_time


    # Realizar previsões no conjunto de teste
    start_forecast_time = time.time()
    test_predict = make_predictions_CNN(model, X_test)
    end_forecast_time = time.time()
    forecast_time = end_forecast_time - start_forecast_time


    # Reverter a normalização das previsões e dos valores reais
    test_predict_original = scaler.inverse_transform(test_predict).flatten()
    y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

    # Obter os índices reais para treino e teste
    train_index = df_loja_grouped.index[:train_size]  # Índices do conjunto de treino
    test_index = df_loja_grouped.index[train_size:]   # Índices do conjunto de teste

    # Dados de treino original
    train_original = scaler.inverse_transform(data_scaled[:train_size]).flatten()

    # Dados reais para plotagem do teste
    data_test_original = scaler.inverse_transform(data_scaled[train_size:]).flatten()

    df_resultado_loja = pd.concat([df_resultado_loja, pd.DataFrame({
        'Data': test_index,
        'Loja': loja,
        'Real': y_test_original,
        'Previsto': test_predict_original
    })])

    df_resultado_tempo = pd.concat([df_resultado_tempo, pd.DataFrame({
        'Loja': [loja],
        'Treino': [train_time],
        'Previsao': [forecast_time]
    })], ignore_index=True)

        # Add traces for train, test, and forecast
    fig.add_trace(go.Scatter(x=train_index, y=train_original, mode='lines', name='Treino', line=dict(color='blue')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name='Previsto', line=dict(color='green')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_index, y=data_test_original, mode='lines', name='Real', line=dict(color='orange')), row=i//2+1, col=i%2+1)
    
    # Update layout for each subplot
    fig.update_xaxes(title_text='Data', row=i//2+1, col=i%2+1)
    fig.update_yaxes(title_text='Vendas', row=i//2+1, col=i%2+1)

# Update layout for the entire figure
fig.update_layout(
    title='Sales Forecast for Each Store',
    height=1500,
    showlegend=False,
    template='plotly_white'
)

# Salvar o dataframe em um arquivo CSV

df_resultado_loja.to_csv('Resultados/Previsao/Favorita/Loja/CNN.csv', index=False)
df_resultado_tempo.to_csv('Resultados/Tempo/Favorita/Loja/CNN.csv', index=False)

# Show the figure
fig.show()


3/3 [==============================] - 0s 0s/step


In [8]:
# Dicionários para armazenar os modelos e tempos para cada loja e item
modelos_por_loja_item = {}
tempos_treinamento = {}
tempos_previsao = {}

# Criar dataframes para armazenar os resultados
df_resultado_loja_item = pd.DataFrame(columns=['Data', 'Loja', 'Item', 'Real', 'Previsto'])
df_resultado_tempo_item = pd.DataFrame(columns=['Loja', 'Item', 'Treino', 'Previsao'])

# Iterar sobre cada loja relevante
for loja in lojas_relevantes_favorita:
    # Pegar os itens relevantes da loja atual
    itens_relevantes = produtos_relevantes_favorita.get(loja, [])
    
    # Criar figura específica para a loja
    fig_loja = make_subplots(
        rows=len(itens_relevantes),
        cols=1,
        subplot_titles=[f'Loja {loja} - Item {item}' for item in itens_relevantes]
    )
    
    for idx, item in enumerate(itens_relevantes):

        df_loja_item = favorita[(favorita['Loja'] == loja) & (favorita['Produto'] == item)]
        df_loja_item = df_loja_item.groupby('Data')['Vendas'].sum().reset_index()
        df_loja_item.set_index('Data', inplace=True)
        data_scaled, scaler = prepare_data_CNN(df_loja_item)
        look_back = 10

        X_train, X_test, y_train, y_test, train_size = split_data_CNN(data_scaled, look_back)

        # Construir e treinar o modelo
        model = build_model_CNN(look_back)

        start_train_time = time.time()
        history = train_model_CNN(model, X_train, y_train)
        end_train_time = time.time()
        train_time = end_train_time - start_train_time


        # Realizar previsões no conjunto de teste
        start_forecast_time = time.time()
        test_predict = make_predictions_CNN(model, X_test)
        end_forecast_time = time.time()
        forecast_time = end_forecast_time - start_forecast_time


        # Reverter a normalização das previsões e dos valores reais
        test_predict_original = scaler.inverse_transform(test_predict).flatten()
        y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

        # Obter os índices reais para treino e teste
        train_index = df_loja_item.index[:train_size]  # Índices do conjunto de treino
        test_index = df_loja_item.index[train_size:]   # Índices do conjunto de teste

        # Dados de treino original
        train_original = scaler.inverse_transform(data_scaled[:train_size]).flatten()

        # Dados reais para plotagem do teste
        data_test_original = scaler.inverse_transform(data_scaled[train_size:]).flatten()

        df_resultado_loja_item = pd.concat([df_resultado_loja_item, pd.DataFrame({
            'Data': test_index,
            'Loja': loja,
            'Item': item,
            'Real': y_test_original,
            'Previsto': test_predict_original
        })])

        df_resultado_tempo_item = pd.concat([df_resultado_tempo_item, pd.DataFrame({
            'Loja': [loja],
            'Item': [item],
            'Treino': [train_time],
            'Previsao': [forecast_time]
        })], ignore_index=True)
        
        # Add traces for train, test, and forecast
        fig_loja.add_trace(go.Scatter(x=train_index, y=train_original, mode='lines', name='Treino', line=dict(color='blue')), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name='Previsto', line=dict(color='green')), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name='Real', line=dict(color='orange')), row=idx+1, col=1)
        
        # Update layout for each subplot
        fig_loja.update_xaxes(title_text='Data', row=idx+1, col=1)
        fig_loja.update_yaxes(title_text='Vendas', row=idx+1, col=1)

    # Update layout for the entire figure
    fig_loja.update_layout(
        title=f'Sales Forecast for Store {loja}',
        height=300 * len(itens_relevantes),
        showlegend=False,
        template='plotly_white'
    )
    # Show the figure
    fig_loja.show()

df_resultado_loja_item.to_csv('Resultados/Previsao/Favorita/Item/CNN.csv', index=False)
df_resultado_tempo_item.to_csv('Resultados/Tempo/Favorita/Item/CNN.csv', index=False)

2/2 [==============================] - 0s 0s/step


3/3 [==============================] - 0s 17ms/step


2/2 [==============================] - 0s 0s/step


2/2 [==============================] - 0s 0s/step


2/2 [==============================] - 0s 0s/step


3/3 [==============================] - 0s 0s/step


2/2 [==============================] - 0s 0s/step


2/2 [==============================] - 0s 541us/step


3/3 [==============================] - 0s 9ms/step


3/3 [==============================] - 0s 1ms/step


# M5

In [9]:
file_path_m5 = 'Bases Tratadas/m5_item_loja_semana.parquet'
df_m5 = pd.read_parquet(file_path_m5)

# Agrupamento de vendas por data
m5_agrupado = df_m5[['Data', 'Vendas']].groupby('Data').sum().reset_index()

# Conversão de Data para o tipo datetime e definição como índice
m5_agrupado.set_index('Data', inplace=True)

In [10]:
# Preparar os dados
data_scaled, scaler = prepare_data_CNN(m5_agrupado)

look_back = 10

# Criar sequências e dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test, train_size = split_data_CNN(data_scaled, look_back)

# Construir e treinar o modelo
model = build_model_CNN(look_back)

start_train_time = time.time()
history = train_model_CNN(model, X_train, y_train)
end_train_time = time.time()
train_time = end_train_time - start_train_time


# Realizar previsões no conjunto de teste
start_forecast_time = time.time()
test_predict = make_predictions_CNN(model, X_test)
end_forecast_time = time.time()
forecast_time = end_forecast_time - start_forecast_time


# Reverter a normalização das previsões e dos valores reais
test_predict_original = scaler.inverse_transform(test_predict).flatten()
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

# Obter os índices reais para treino e teste
train_index = m5_agrupado.index[:train_size]  # Índices do conjunto de treino
test_index = m5_agrupado.index[train_size:]   # Índices do conjunto de teste

# Dados de treino original
train_original = scaler.inverse_transform(data_scaled[:train_size]).flatten()

# Dados reais para plotagem do teste
data_test_original = scaler.inverse_transform(data_scaled[train_size:]).flatten()

# Plotando os dados originais e previstos
fig = go.Figure()

# Dados reais (treino)
fig.add_trace(go.Scatter(
    x=train_index, 
    y=train_original, 
    mode='lines', 
    name='Treino (Real)', 
    line=dict(color='blue')
))

# Dados reais (teste)
fig.add_trace(go.Scatter(
    x=test_index, 
    y=data_test_original, 
    mode='lines', 
    name='Teste (Real)', 
    line=dict(color='orange')
))

# Dados previstos (teste)
fig.add_trace(go.Scatter(
    x=test_index[:len(test_predict_original)],  # Garantir o alinhamento
    y=test_predict_original, 
    mode='lines', 
    name='Teste (Previsto)', 
    line=dict(color='green')
))

# Configuração do gráfico
fig.update_layout(
    title='Vendas Reais vs Previstas',
    xaxis_title='Data',
    yaxis_title='Vendas',
    legend_title='Legenda',
    template='plotly_white'
)

fig.show()

3/3 [==============================] - 0s 2ms/step


In [11]:
# Criar um dataframe com data, valor real e valor previsto
df_resultado = pd.DataFrame({
    'Data': test_index,
    'Real': y_test_original,
    'Previsto': test_predict_original
})

# Salvar o dataframe em um arquivo CSV
df_resultado.to_csv('Resultados/Previsao/M5/Tudo/CNN.csv', index=False)

# Criar um dataframe com o tempo de treino e previsão
df_tempo = pd.DataFrame({
    'Treino': [train_time],
    'Previsao': [forecast_time]
})
# Salvar o dataframe em um arquivo CSV
df_tempo.to_csv('Resultados/Tempo/M5/Tudo/CNN.csv', index=False)

In [12]:
lojas_relevantes_m5 = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']

produtos_relevantes_m5 = {
    'CA_1': ['FOODS_3_090', 'FOODS_3_586'],
    'CA_2': ['FOODS_3_586', 'FOODS_3_252'],
    'CA_3': ['FOODS_3_090', 'FOODS_3_586'],
    'CA_4': ['FOODS_3_090', 'FOODS_3_586'],
    'TX_1': ['FOODS_3_586', 'FOODS_3_090'],
    'TX_2': ['FOODS_3_586', 'FOODS_3_090'],
    'TX_3': ['FOODS_3_586', 'FOODS_3_090'],
    'WI_1': ['FOODS_3_226', 'FOODS_3_694'],
    'WI_2': ['FOODS_3_226', 'FOODS_3_694'],
    'WI_3': ['FOODS_3_090', 'FOODS_3_226']
}

In [13]:
modelos_por_loja = {}
tempos_treinamento = {}
tempos_previsao = {}

# Create a figure
fig = make_subplots(rows=5, cols=2, subplot_titles=[f'Loja {loja}' for loja in lojas_relevantes_m5])

df_resultado_loja = pd.DataFrame(columns=['Data', 'Loja', 'Real', 'Previsto'])
df_resultado_tempo = pd.DataFrame(columns=['Loja', 'Treino', 'Previsao'])

for i, loja in enumerate(lojas_relevantes_m5):

    df_loja = df_m5[df_m5['Loja'] == loja]
    df_loja_grouped = df_loja.groupby('Data')['Vendas'].sum().reset_index()
    df_loja_grouped.set_index('Data', inplace=True)
    data_scaled, scaler = prepare_data_CNN(df_loja_grouped)
    look_back = 10

    X_train, X_test, y_train, y_test, train_size = split_data_CNN(data_scaled, look_back)

    # Construir e treinar o modelo
    model = build_model_CNN(look_back)

    start_train_time = time.time()
    history = train_model_CNN(model, X_train, y_train)
    end_train_time = time.time()
    train_time = end_train_time - start_train_time


    # Realizar previsões no conjunto de teste
    start_forecast_time = time.time()
    test_predict = make_predictions_CNN(model, X_test)
    end_forecast_time = time.time()
    forecast_time = end_forecast_time - start_forecast_time


    # Reverter a normalização das previsões e dos valores reais
    test_predict_original = scaler.inverse_transform(test_predict).flatten()
    y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

    # Obter os índices reais para treino e teste
    train_index = df_loja_grouped.index[:train_size]  # Índices do conjunto de treino
    test_index = df_loja_grouped.index[train_size:]   # Índices do conjunto de teste

    # Dados de treino original
    train_original = scaler.inverse_transform(data_scaled[:train_size]).flatten()

    # Dados reais para plotagem do teste
    data_test_original = scaler.inverse_transform(data_scaled[train_size:]).flatten()

    df_resultado_loja = pd.concat([df_resultado_loja, pd.DataFrame({
        'Data': test_index,
        'Loja': loja,
        'Real': y_test_original,
        'Previsto': test_predict_original
    })])

    df_resultado_tempo = pd.concat([df_resultado_tempo, pd.DataFrame({
        'Loja': [loja],
        'Treino': [train_time],
        'Previsao': [forecast_time]
    })], ignore_index=True)

        # Add traces for train, test, and forecast
    fig.add_trace(go.Scatter(x=train_index, y=train_original, mode='lines', name='Treino', line=dict(color='blue')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name='Previsto', line=dict(color='green')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_index, y=data_test_original, mode='lines', name='Real', line=dict(color='orange')), row=i//2+1, col=i%2+1)
    
    # Update layout for each subplot
    fig.update_xaxes(title_text='Data', row=i//2+1, col=i%2+1)
    fig.update_yaxes(title_text='Vendas', row=i//2+1, col=i%2+1)

# Update layout for the entire figure
fig.update_layout(
    title='Sales Forecast for Each Store',
    height=1500,
    showlegend=False,
    template='plotly_white'
)

# Salvar o dataframe em um arquivo CSV

df_resultado_loja.to_csv('Resultados/Previsao/M5/Loja/CNN.csv', index=False)
df_resultado_tempo.to_csv('Resultados/Tempo/M5/Loja/CNN.csv', index=False)

# Show the figure
fig.show()

3/3 [==============================] - 0s 4ms/step


In [14]:
# Dicionários para armazenar os modelos e tempos para cada loja e item
modelos_por_loja_item = {}
tempos_treinamento = {}
tempos_previsao = {}

# Criar dataframes para armazenar os resultados
df_resultado_loja_item = pd.DataFrame(columns=['Data', 'Loja', 'Item', 'Real', 'Previsto'])
df_resultado_tempo_item = pd.DataFrame(columns=['Loja', 'Item', 'Treino', 'Previsao'])

# Iterar sobre cada loja relevante
for loja in lojas_relevantes_m5:
    # Pegar os itens relevantes da loja atual
    itens_relevantes = produtos_relevantes_m5.get(loja, [])
    
    # Criar figura específica para a loja
    fig_loja = make_subplots(
        rows=len(itens_relevantes),
        cols=1,
        subplot_titles=[f'Loja {loja} - Item {item}' for item in itens_relevantes]
    )
    
    for idx, item in enumerate(itens_relevantes):

        df_loja_item = df_m5[(df_m5['Loja'] == loja) & (df_m5['Produto'] == item)]
        df_loja_item = df_loja_item.groupby('Data')['Vendas'].sum().reset_index()
        df_loja_item.set_index('Data', inplace=True)
        data_scaled, scaler = prepare_data_CNN(df_loja_item)
        look_back = 10

        X_train, X_test, y_train, y_test, train_size = split_data_CNN(data_scaled, look_back)

        # Construir e treinar o modelo
        model = build_model_CNN(look_back)

        start_train_time = time.time()
        history = train_model_CNN(model, X_train, y_train)
        end_train_time = time.time()
        train_time = end_train_time - start_train_time


        # Realizar previsões no conjunto de teste
        start_forecast_time = time.time()
        test_predict = make_predictions_CNN(model, X_test)
        end_forecast_time = time.time()
        forecast_time = end_forecast_time - start_forecast_time


        # Reverter a normalização das previsões e dos valores reais
        test_predict_original = scaler.inverse_transform(test_predict).flatten()
        y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

        # Obter os índices reais para treino e teste
        train_index = df_loja_item.index[:train_size]  # Índices do conjunto de treino
        test_index = df_loja_item.index[train_size:]   # Índices do conjunto de teste

        # Dados de treino original
        train_original = scaler.inverse_transform(data_scaled[:train_size]).flatten()

        # Dados reais para plotagem do teste
        data_test_original = scaler.inverse_transform(data_scaled[train_size:]).flatten()

        df_resultado_loja_item = pd.concat([df_resultado_loja_item, pd.DataFrame({
            'Data': test_index,
            'Loja': loja,
            'Item': item,
            'Real': y_test_original,
            'Previsto': test_predict_original
        })])

        df_resultado_tempo_item = pd.concat([df_resultado_tempo_item, pd.DataFrame({
            'Loja': [loja],
            'Item': [item],
            'Treino': [train_time],
            'Previsao': [forecast_time]
        })], ignore_index=True)
        
        # Add traces for train, test, and forecast
        fig_loja.add_trace(go.Scatter(x=train_index, y=train_original, mode='lines', name='Treino', line=dict(color='blue')), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name='Previsto', line=dict(color='green')), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name='Real', line=dict(color='orange')), row=idx+1, col=1)
        
        # Update layout for each subplot
        fig_loja.update_xaxes(title_text='Data', row=idx+1, col=1)
        fig_loja.update_yaxes(title_text='Vendas', row=idx+1, col=1)

    # Update layout for the entire figure
    fig_loja.update_layout(
        title=f'Sales Forecast for Store {loja}',
        height=300 * len(itens_relevantes),
        showlegend=False,
        template='plotly_white'
    )
    # Show the figure
    fig_loja.show()

df_resultado_loja_item.to_csv('Resultados/Previsao/M5/Item/CNN.csv', index=False)
df_resultado_tempo_item.to_csv('Resultados/Tempo/M5/Item/CNN.csv', index=False)

3/3 [==============================] - 0s 2ms/step


3/3 [==============================] - 0s 0s/step


3/3 [==============================] - 0s 0s/step


3/3 [==============================] - 0s 0s/step


3/3 [==============================] - 0s 0s/step


3/3 [==============================] - 0s 8ms/step


3/3 [==============================] - 0s 0s/step


3/3 [==============================] - 0s 0s/step


3/3 [==============================] - 0s 0s/step


3/3 [==============================] - 0s 0s/step


# OLIST

In [15]:
file_path_olist = 'Bases Tratadas/olist_item_loja_semana.parquet'
df_olist = pd.read_parquet(file_path_olist)
# Agrupamento de vendas por data
olist_agrupado = df_olist[['Data', 'Vendas']].groupby('Data').sum().reset_index()

# Conversão de Data para o tipo datetime e definição como índice
olist_agrupado.set_index('Data', inplace=True)

In [16]:
# Preparar os dados
data_scaled, scaler = prepare_data_CNN(olist_agrupado)

look_back = 10

# Criar sequências e dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test, train_size = split_data_CNN(data_scaled, look_back)

# Construir e treinar o modelo
model = build_model_CNN(look_back)

start_train_time = time.time()
history = train_model_CNN(model, X_train, y_train)
end_train_time = time.time()
train_time = end_train_time - start_train_time


# Realizar previsões no conjunto de teste
start_forecast_time = time.time()
test_predict = make_predictions_CNN(model, X_test)
end_forecast_time = time.time()
forecast_time = end_forecast_time - start_forecast_time


# Reverter a normalização das previsões e dos valores reais
test_predict_original = scaler.inverse_transform(test_predict).flatten()
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

# Obter os índices reais para treino e teste
train_index = olist_agrupado.index[:train_size]  # Índices do conjunto de treino
test_index = olist_agrupado.index[train_size:]   # Índices do conjunto de teste

# Dados de treino original
train_original = scaler.inverse_transform(data_scaled[:train_size]).flatten()

# Dados reais para plotagem do teste
data_test_original = scaler.inverse_transform(data_scaled[train_size:]).flatten()

# Plotando os dados originais e previstos
fig = go.Figure()

# Dados reais (treino)
fig.add_trace(go.Scatter(
    x=train_index, 
    y=train_original, 
    mode='lines', 
    name='Treino (Real)', 
    line=dict(color='blue')
))

# Dados reais (teste)
fig.add_trace(go.Scatter(
    x=test_index, 
    y=data_test_original, 
    mode='lines', 
    name='Teste (Real)', 
    line=dict(color='orange')
))

# Dados previstos (teste)
fig.add_trace(go.Scatter(
    x=test_index[:len(test_predict_original)],  # Garantir o alinhamento
    y=test_predict_original, 
    mode='lines', 
    name='Teste (Previsto)', 
    line=dict(color='green')
))

# Configuração do gráfico
fig.update_layout(
    title='Vendas Reais vs Previstas',
    xaxis_title='Data',
    yaxis_title='Vendas',
    legend_title='Legenda',
    template='plotly_white'
)

fig.show()

1/1 [==============================] - 0s 100ms/step


In [17]:
# Criar um dataframe com data, valor real e valor previsto
df_resultado = pd.DataFrame({
    'Data': test_index,
    'Real': y_test_original,
    'Previsto': test_predict_original
})

# Salvar o dataframe em um arquivo CSV
df_resultado.to_csv('Resultados/Previsao/Olist/Tudo/CNN.csv', index=False)

# Criar um dataframe com o tempo de treino e previsão
df_tempo = pd.DataFrame({
    'Treino': [train_time],
    'Previsao': [forecast_time]
})
# Salvar o dataframe em um arquivo CSV
df_tempo.to_csv('Resultados/Tempo/Olist/Tudo/CNN.csv', index=False)

In [18]:
lojas_relevantes_olist = [
    '6560211a19b47992c3666cc44a7e94c0', '4a3ca9315b744ce9f8e9374361493884', 
    '1f50f920176fa81dab994f9023523100', 'cc419e0650a3c5ba77189a1882b7556a', 
    'da8622b14eb17ae2831f4ac5b9dab84a', '955fee9216a65b617aa5c0531780ce60', 
    '1025f0e2d44d7041d6cf58b6550e0bfa', '7c67e1448b00f6e969d365cea6b010ab', 
    'ea8482cd71df3c1969d7b9473ff13abc', '7a67c85e85bb2ce8582c35f2203ad736'
]

produtos_relevantes_olist = {
    '6560211a19b47992c3666cc44a7e94c0': ['a92930c327948861c015c919a0bcb4a8', 'd017a2151d543a9885604dc62a3d9dcc'],
    '4a3ca9315b744ce9f8e9374361493884': ['99a4788cb24856965c36a24e339b6058', '35afc973633aaeb6b877ff57b2793310'],
    '1f50f920176fa81dab994f9023523100': ['422879e10f46682990de24d770e7f83d', '389d119b48cf3043d311335e499d9c6b'],
    'cc419e0650a3c5ba77189a1882b7556a': ['154e7e31ebfa092203795c972e5804a6', '2b4609f8948be18874494203496bc318'],
    'da8622b14eb17ae2831f4ac5b9dab84a': ['e0cf79767c5b016251fe139915c59a26', 'fc1d8637c0268af3db482c14b7ef8e75'],
    '955fee9216a65b617aa5c0531780ce60': ['aca2eb7d00ea1a7b8ebd4e68314663af', '54d9ac713e253fa1fae9c8003b011c2a'],
    '1025f0e2d44d7041d6cf58b6550e0bfa': ['b532349fe46b38fbc7bb3914c1bdae07', '9ecadb84c81da840dbf3564378b586e9'],
    '7c67e1448b00f6e969d365cea6b010ab': ['8ed094bfe076c568f6bb10feada3f75d', '3eef0cb94ba82de806bb30ab743c7655'],
    'ea8482cd71df3c1969d7b9473ff13abc': ['97017430754804328eb9597b7f85da03', 'c1f5307decb89342351bec53668cffd9'],
    '7a67c85e85bb2ce8582c35f2203ad736': ['c6dd917a0be2a704582055949915ab32', '601a360bd2a916ecef0e88de72a6531a']
}

In [19]:
modelos_por_loja = {}
tempos_treinamento = {}
tempos_previsao = {}

# Create a figure
fig = make_subplots(rows=5, cols=2, subplot_titles=[f'Loja {loja}' for loja in lojas_relevantes_olist])

df_resultado_loja = pd.DataFrame(columns=['Data', 'Loja', 'Real', 'Previsto'])
df_resultado_tempo = pd.DataFrame(columns=['Loja', 'Treino', 'Previsao'])

for i, loja in enumerate(lojas_relevantes_olist):

    df_loja = df_olist[df_olist['Loja'] == loja]
    df_loja_grouped = df_loja.groupby('Data')['Vendas'].sum().reset_index()
    date_range = pd.date_range(start=df_olist['Data'].min(), end=df_olist['Data'].max(), freq='W')
    date_range = date_range + pd.DateOffset(days=1)
    df_loja_grouped = df_loja_grouped.set_index('Data').reindex(date_range, fill_value=0).rename_axis('Data').reset_index()
    df_loja_grouped.set_index('Data', inplace=True)
    data_scaled, scaler = prepare_data_CNN(df_loja_grouped)
    look_back = 10

    X_train, X_test, y_train, y_test, train_size = split_data_CNN(data_scaled, look_back)

    # Construir e treinar o modelo
    model = build_model_CNN(look_back)

    start_train_time = time.time()
    history = train_model_CNN(model, X_train, y_train)
    end_train_time = time.time()
    train_time = end_train_time - start_train_time


    # Realizar previsões no conjunto de teste
    start_forecast_time = time.time()
    test_predict = make_predictions_CNN(model, X_test)
    end_forecast_time = time.time()
    forecast_time = end_forecast_time - start_forecast_time


    # Reverter a normalização das previsões e dos valores reais
    test_predict_original = scaler.inverse_transform(test_predict).flatten()
    y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

    # Obter os índices reais para treino e teste
    train_index = df_loja_grouped.index[:train_size]  # Índices do conjunto de treino
    test_index = df_loja_grouped.index[train_size:]   # Índices do conjunto de teste

    # Dados de treino original
    train_original = scaler.inverse_transform(data_scaled[:train_size]).flatten()

    # Dados reais para plotagem do teste
    data_test_original = scaler.inverse_transform(data_scaled[train_size:]).flatten()

    df_resultado_loja = pd.concat([df_resultado_loja, pd.DataFrame({
        'Data': test_index,
        'Loja': loja,
        'Real': y_test_original,
        'Previsto': test_predict_original
    })])

    df_resultado_tempo = pd.concat([df_resultado_tempo, pd.DataFrame({
        'Loja': [loja],
        'Treino': [train_time],
        'Previsao': [forecast_time]
    })], ignore_index=True)

        # Add traces for train, test, and forecast
    fig.add_trace(go.Scatter(x=train_index, y=train_original, mode='lines', name='Treino', line=dict(color='blue')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name='Previsto', line=dict(color='green')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_index, y=data_test_original, mode='lines', name='Real', line=dict(color='orange')), row=i//2+1, col=i%2+1)
    
    # Update layout for each subplot
    fig.update_xaxes(title_text='Data', row=i//2+1, col=i%2+1)
    fig.update_yaxes(title_text='Vendas', row=i//2+1, col=i%2+1)

# Update layout for the entire figure
fig.update_layout(
    title='Sales Forecast for Each Store',
    height=1500,
    showlegend=False,
    template='plotly_white'
)

# Salvar o dataframe em um arquivo CSV

df_resultado_loja.to_csv('Resultados/Previsao/Olist/Loja/CNN.csv', index=False)
df_resultado_tempo.to_csv('Resultados/Tempo/Olist/Loja/CNN.csv', index=False)

# Show the figure
fig.show()

1/1 [==============================] - 0s 92ms/step


In [20]:
# Dicionários para armazenar os modelos e tempos para cada loja e item
modelos_por_loja_item = {}
tempos_treinamento = {}
tempos_previsao = {}

# Criar dataframes para armazenar os resultados
df_resultado_loja_item = pd.DataFrame(columns=['Data', 'Loja', 'Item', 'Real', 'Previsto'])
df_resultado_tempo_item = pd.DataFrame(columns=['Loja', 'Item', 'Treino', 'Previsao'])

# Iterar sobre cada loja relevante
for loja in lojas_relevantes_olist:
    # Pegar os itens relevantes da loja atual
    itens_relevantes = produtos_relevantes_olist.get(loja, [])
    
    # Criar figura específica para a loja
    fig_loja = make_subplots(
        rows=len(itens_relevantes),
        cols=1,
        subplot_titles=[f'Loja {loja} - Item {item}' for item in itens_relevantes]
    )
    
    for idx, item in enumerate(itens_relevantes):

        df_loja_item = df_olist[(df_olist['Loja'] == loja) & (df_olist['Produto'] == item)]
        df_loja_item = df_loja_item.groupby('Data')['Vendas'].sum().reset_index()
        date_range = pd.date_range(start=df_olist['Data'].min(), end=df_olist['Data'].max(), freq='W')
        date_range = date_range + pd.DateOffset(days=1)
        df_loja_item = df_loja_item.set_index('Data').reindex(date_range, fill_value=0).rename_axis('Data').reset_index()
        df_loja_item.set_index('Data', inplace=True)

        try:
            data_scaled, scaler = prepare_data_CNN(df_loja_item)
            look_back = 10
        

            X_train, X_test, y_train, y_test, train_size = split_data_CNN(data_scaled, look_back)

            # Construir e treinar o modelo
            model = build_model_CNN(look_back)

            start_train_time = time.time()
            history = train_model_CNN(model, X_train, y_train)
            end_train_time = time.time()
            train_time = end_train_time - start_train_time


            # Realizar previsões no conjunto de teste
            start_forecast_time = time.time()
            test_predict = make_predictions_CNN(model, X_test)
            end_forecast_time = time.time()
            forecast_time = end_forecast_time - start_forecast_time


            # Reverter a normalização das previsões e dos valores reais
            test_predict_original = scaler.inverse_transform(test_predict).flatten()
            y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

            # Obter os índices reais para treino e teste
            train_index = df_loja_item.index[:train_size]  # Índices do conjunto de treino
            test_index = df_loja_item.index[train_size:]   # Índices do conjunto de teste

            # Dados de treino original
            train_original = scaler.inverse_transform(data_scaled[:train_size]).flatten()

            # Dados reais para plotagem do teste
            data_test_original = scaler.inverse_transform(data_scaled[train_size:]).flatten()

            df_resultado_loja_item = pd.concat([df_resultado_loja_item, pd.DataFrame({
                'Data': test_index,
                'Loja': loja,
                'Item': item,
                'Real': y_test_original,
                'Previsto': test_predict_original
            })])

            df_resultado_tempo_item = pd.concat([df_resultado_tempo_item, pd.DataFrame({
                'Loja': [loja],
                'Item': [item],
                'Treino': [train_time],
                'Previsao': [forecast_time]
            })], ignore_index=True)
            
            # Add traces for train, test, and forecast
            fig_loja.add_trace(go.Scatter(x=train_index, y=train_original, mode='lines', name='Treino', line=dict(color='blue')), row=idx+1, col=1)
            fig_loja.add_trace(go.Scatter(x=test_index, y=test_predict_original, mode='lines', name='Previsto', line=dict(color='green')), row=idx+1, col=1)
            fig_loja.add_trace(go.Scatter(x=test_index, y=y_test_original, mode='lines', name='Real', line=dict(color='orange')), row=idx+1, col=1)
            
            # Update layout for each subplot
            fig_loja.update_xaxes(title_text='Data', row=idx+1, col=1)
            fig_loja.update_yaxes(title_text='Vendas', row=idx+1, col=1)
        except:
            continue

    # Update layout for the entire figure
    fig_loja.update_layout(
        title=f'Sales Forecast for Store {loja}',
        height=300 * len(itens_relevantes),
        showlegend=False,
        template='plotly_white'
    )
    # Show the figure
    fig_loja.show()

df_resultado_loja_item.to_csv('Resultados/Previsao/Olist/Item/CNN.csv', index=False)
df_resultado_tempo_item.to_csv('Resultados/Tempo/Olist/Item/CNN.csv', index=False)

1/1 [==============================] - 0s 95ms/step


1/1 [==============================] - 0s 100ms/step


1/1 [==============================] - 0s 99ms/step


1/1 [==============================] - 0s 84ms/step


1/1 [==============================] - 0s 84ms/step


1/1 [==============================] - 0s 100ms/step


1/1 [==============================] - 0s 100ms/step


1/1 [==============================] - 0s 89ms/step


1/1 [==============================] - 0s 100ms/step


1/1 [==============================] - 0s 88ms/step
